In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import re

In [2]:
df = pd.read_excel('./01_raw_data/range/친환경차_조회_2025-08-05.xlsx',sheet_name='전기자동차', parse_dates=['적용일자'])
df[['축전지정격전압(V)', '축전지정격용량(Ah)']] = (
    df['축전지정격전압/용량(V/Ah)']
      .str.split('/', expand=True)
)
df[['축전지정격전압(V)', '축전지정격용량(Ah)']] = df[['축전지정격전압(V)', '축전지정격용량(Ah)']].apply(pd.to_numeric, errors='coerce')
df= df.drop(columns=['축전지정격전압/용량(V/Ah)','제외일자', '구동방식','축전지정격전압(V)'])
df

c:\Users\junjo\anaconda3\envs\ev_project2\lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,제작사 또는 수입사,모델명,에너지소비효율(km/kWh),1회충전주행거리(km),타이어형식(전),적용일자,축전지정격용량(Ah)
0,기아,쏘울 전기자동차(30kWh),5.2,180.0,205/60R16,2019-11-20,80.0
1,기아,쏘울 전기자동차(39.2kWh),5.6,250.0,215/55R17,2019-03-15,120.0
2,기아,쏘울 전기자동차(64kWh),5.4,386.0,215/55R17,2019-03-15,180.0
3,기아,니로 전기자동차,5.3,385.0,215/55R17,2018-07-20,180.0
4,기아,니로 전기자동차,5.5,246.0,215/55R17,2018-07-20,120.0
...,...,...,...,...,...,...,...
633,닛산,LEAF (23.8kWh),5.0,131.0,215/50R17,2014-12-22,66.0
634,한국GM,스파크 EV,5.6,135.0,185/55R15,2013-09-06,60.0
635,한국GM,스파크 EV,6.0,128.0,185/55R15,2013-09-06,52.0
636,재규어랜드로버,재규어 I-PACE EV400,3.5,333.0,245/50R20,2019-02-20,232.0


In [3]:
def classify_tire_usage(tire):
    """
    타이어 문자열 기반 용도 분류
    - micro: 림 ≤ 13인치 (경차/초소형)
    - commercial: LT, RxxC, PR 포함
    - passenger: 그 외
    """
    s = re.sub(r'[\s\-/]', '', str(tire)).upper()

    # 상업용 판정
    if s.startswith('LT'):
        return 'commercial'
    if re.search(r'R\d{2}C', s):
        return 'commercial'
    if re.search(r'\d+PR', s):
        return 'commercial'

    # 초소형 차 판정
    m = re.search(r'R(\d{2})', s)
    if m and int(m.group(1)) <= 13:
        return 'micro'

    # 3) 나머지 승용
    return 'passenger'
output_rows = []
start_date = datetime(2018, 1, 1)
end_date   = datetime(2024, 12, 31)
active = []  

current = start_date
while current <= end_date:
    today = df[df['적용일자'] == current]
    active.extend(today.to_dict('records'))

    cutoff = current - timedelta(days=365)
    active = [r for r in active if r['적용일자'] > cutoff]
    
    comm = [r for r in active if classify_tire_usage(r['타이어형식(전)']) == 'commercial']
    nonc = [r for r in active if classify_tire_usage(r['타이어형식(전)']) == 'passenger']
    
    def mean_of(key, records):
        return pd.Series([r[key] for r in records]).mean() if records else None
    
    avg_eff_all   = mean_of('에너지소비효율(km/kWh)', active)
    avg_range_all = mean_of('1회충전주행거리(km)',    active)
    avg_cap_all   = mean_of('축전지정격용량(Ah)',      active)

    avg_eff_comm   = mean_of('에너지소비효율(km/kWh)', comm)
    avg_range_comm = mean_of('1회충전주행거리(km)',    comm)
    avg_cap_comm   = mean_of('축전지정격용량(Ah)',      comm)
    
    avg_eff_non   = mean_of('에너지소비효율(km/kWh)', nonc)
    avg_range_non = mean_of('1회충전주행거리(km)',    nonc)
    avg_cap_non   = mean_of('축전지정격용량(Ah)',      nonc)
    
    output_rows.append({
        '날짜': current.strftime('%Y-%m-%d'),
        # 전체
        '평균_에너지소비효율_전체':        avg_eff_all,
        '평균_1회충전주행거리_전체(km)':  avg_range_all,
        '평균_충전지용량_전체(Ah)':        avg_cap_all,
        # 상업용
        '평균_에너지소비효율_상업용':       avg_eff_comm,
        '평균_1회충전주행거리_상업용(km)': avg_range_comm,
        '평균_충전지용량_상업용(Ah)':       avg_cap_comm,
        # 비상업용
        '평균_에너지소비효율_비상업용':     avg_eff_non,
        '평균_1회충전주행거리_비상업용(km)': avg_range_non,
        '평균_충전지용량_비상업용(Ah)':     avg_cap_non,
    })
    
    current += timedelta(days=1)

df_daily = pd.DataFrame(output_rows)
df_daily

,날짜,평균_에너지소비효율_전체,평균_1회충전주행거리_전체(km),평균_충전지용량_전체(Ah),평균_에너지소비효율_상업용,평균_1회충전주행거리_상업용(km),평균_충전지용량_상업용(Ah),평균_에너지소비효율_비상업용,평균_1회충전주행거리_비상업용(km),평균_충전지용량_비상업용(Ah)
0,2018-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2552,2024-12-27,4.380292,386.751825,171.960741,3.057143,208.785714,158.138462,4.516393,409.52459,173.726446
2553,2024-12-28,4.380292,386.751825,171.960741,3.057143,208.785714,158.138462,4.516393,409.52459,173.726446
2554,2024-12-29,4.380292,386.751825,171.960741,3.057143,208.785714,158.138462,4.516393,409.52459,173.726446
2555,2024-12-30,4.380292,386.751825,171.960741,3.057143,208.785714,158.138462,4.516393,409.52459,173.726446


In [6]:

df_daily['날짜'] = pd.to_datetime(df_daily['날짜'])
df_daily['연월'] = df_daily['날짜'].dt.to_period('M')

monthly = (
    df_daily
    .groupby('연월')
    .mean(numeric_only=True)
    .reset_index()
)

monthly['연월'] = monthly['연월'].dt.to_timestamp()
monthly['year'] = monthly['연월'].dt.year
monthly['month'] = monthly['연월'].dt.month
monthly['차종'] = None

monthly_long = pd.DataFrame()

mapping = {
    '평균_에너지소비효율_전체':        ('소계', '연료소비효율'),
    '평균_1회충전주행거리_전체(km)':  ('소계', '1회충전주행거리'),
    '평균_충전지용량_전체(Ah)':        ('소계', '배터리충전용량'),
    '평균_에너지소비효율_상업용':       ('화물', '연료소비효율'),
    '평균_1회충전주행거리_상업용(km)': ('화물', '1회충전주행거리'),
    '평균_충전지용량_상업용(Ah)':       ('화물', '배터리충전용량'),
    '평균_에너지소비효율_비상업용':     ('승용', '연료소비효율'),
    '평균_1회충전주행거리_비상업용(km)': ('승용', '1회충전주행거리'),
    '평균_충전지용량_비상업용(Ah)':     ('승용', '배터리충전용량'),
}

for vehicle_type in ['소계', '화물', '승용']:
    temp = monthly[['year', 'month']].copy()
    temp['차종'] = vehicle_type
    temp['연료소비효율'] = monthly[[k for k,v in mapping.items() if v==(vehicle_type,'연료소비효율')]].iloc[:,0]
    temp['1회충전주행거리'] = monthly[[k for k,v in mapping.items() if v==(vehicle_type,'1회충전주행거리')]].iloc[:,0]
    temp['배터리충전용량'] = monthly[[k for k,v in mapping.items() if v==(vehicle_type,'배터리충전용량')]].iloc[:,0]
    monthly_long = pd.concat([monthly_long, temp], ignore_index=True)

monthly_long = monthly_long[['year', 'month', '차종', '연료소비효율', '1회충전주행거리', '배터리충전용량']]

monthly_long.to_csv('./02_interim_data/ev_data_monthly_average.csv', index=False, encoding='utf-8-sig')
monthly_long.tail(50)


,year,month,차종,연료소비효율,1회충전주행거리,배터리충전용량
202,2020,11,승용,4.560000,255.733333,129.462857
203,2020,12,승용,4.500000,262.833333,134.100000
204,2021,1,승용,4.500000,262.833333,134.100000
205,2021,2,승용,4.500000,262.833333,134.100000
206,2021,3,승용,4.528226,271.423387,132.795968
207,2021,4,승용,4.625000,300.875000,128.325000
208,2021,5,승용,4.667476,339.120968,147.515261
209,2021,6,승용,4.775079,354.550794,152.591111
210,2021,7,승용,4.919497,377.172896,149.084390
211,2021,8,승용,4.898704,386.093437,153.248324
